This notebook is associated with the paper "The relative class number one problem for function fields, II" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and Magma (tested using version 2.26-9); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we make computations as part of the proof that for $q=2$, there are no non-Galois covers with relative class number 1. This depends on the exhaustion over Weil polynomials performed in "The relative class number one problem for function fields, I".

In [311]:
load("auxiliary.sage")

In [312]:
P.<T> = QQ[]

Read data about the exhaustion from an Excel spreadsheet.

In [313]:
import pandas

In [314]:
df = pandas.read_excel('polys.xlsx')
assert 'Cyclic' in df
print(list(df))

['Unnamed: 0', 'd', 'g', "g'", 'Label of J(C)', '#J(C)', 'Counts of C', "Counts of C'", 'Cyclic']


In [315]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    counts1 = eval(r["Counts of C"])
    counts2 = eval(r["Counts of C'"])
    ordj = r["#J(C)"]
    candidates[d,g,g1].append((counts1, counts2, ordj))

Identify all Weil polynomials with specified initial Frobenius traces.

In [316]:
def weil_polys_from_traces(P, d, q, l):
    Q.<t> = PowerSeriesRing(QQ)
    u = sum(-l[i-1]*t^i/i for i in range(1,len(l)+1))
    v = exp(u)
    l2 = v.polynomial().list()[:len(l)+1]
    l2 = [ZZ(i) for i in l2]
    l3 = P.weil_polynomials(d=d, q=q, lead=l2)
    if len(l2) < d//2+1:
        return l3
    return [w for w in l3 if w.is_weil_polynomial()]

In [317]:
def weil_polys_from_point_counts(P, d, q, l):
    return weil_polys_from_traces(P, d, q, [q^i+1-l[i-1] for i in range(1, len(l)+1)])

Confirm that for $d>2$, the Galois group contains a $d$-cycle.

In [318]:
for (d, g, g1) in candidates:
    if d > 2:
        for (a,b,_) in candidates[d,g,g1]:
            u = weil_poly_from_point_count(a, g)
            v = weil_poly_from_point_count(b, g1)
            ct1 = place_count_from_weil_poly(u, 5)
            ct2 = place_count_from_weil_poly(v, 5)
            if not (ct1[0] > sum(ct2[i] for i in range((d-1)//2)) + (0 if d%2 else 1/2*ct2[d//2-1])):
                print(d,g,g1,ct1,ct2)

5 2 6 [5, 2, 0, 2, 4] [0, 5, 0, 0, 5]


Compute possible splitting sequences given the point counts for $C$ and $C'$ and the degree $d$ of the morphism.

- `alternating`: if $d$ is odd, assume the Galois group is contained in $A_d$; if $d$ is even, assume the quadratic resolvent is constant.
- `seed`: if specified, only return sequences that extend one in this list. Assumes all specified sequences are of the same length.
- `avoid`: a list of splitting types that must be avoided.

In [424]:
def splitting_sequences(ct1, ct2, d, alternating=False, seed=[[]], avoid=[]):
    n = len(ct1)
    assert n == len(ct2)
    ct1 = place_count_from_point_count(ct1, n)
    parts = Partitions(d)
    parts0 = [k for k in parts if (len(k)-d)%2 == 0]
    parts1 = [k for k in parts if (len(k)-d)%2 == 1]
    l = seed[:]
    for i in range(len(l[0])+1, n+1):
        l2 = []
        for t in l:
            if not alternating:
                tmp = parts
            elif d%2 == 1 or i%2 == 0:
                tmp = parts0
            else:
                tmp = parts1
            tmp = [p for p in tmp if p not in avoid]
            for u in itertools.combinations_with_replacement(tmp, ct1[i-1]):
                t2 = t + [u]
                ct = [0 for j in range(i)]
                for j in range(1, i+1):
                    for k in t2[j-1]:
                        for c in k:
                            for c2 in range(1, i//(j*c) + 1):
                                ct[j*c*c2-1] += j*c
                if ct[i-1] == ct2[i-1]:
                    l2.append(t2)
        l = l2
    return l

Convert a splitting sequence to a count of places in the covering curve.

In [320]:
def place_count_from_splittings(s):
    n = len(s)
    ctp = [0 for _ in range(n)]
    for i in range(n):
        for p in s[i]:
            for j in p:
                if (i+1)*j <= n:
                    ctp[(i+1)*j-1] += 1
    return ctp

Convert a splitting sequence to the traces of the quadratic resolvent. This assumes that the quadratic resolvent is purely geometric.

In [321]:
def splittings_to_quadratic_resolvent(ct1, s):
    n = max(len(ct1), len(s))
    s2 = [tuple([1,1] if (sum(p)-len(p))%2==0 else [2] for p in t) for t in s]
    ctp = place_count_from_splittings(s2)
    ct = point_count_from_place_count(ctp, n)
    return tuple(ct1[i] - ct[i] for i in range(n))

Case $d=3$.

Subcase $(g,g') = (4,10)$.

In [322]:
[(a[0], a[1], b[0], b[1]) for (a,b,_) in candidates[3,4,10]]

[(6, 6, 0, 0), (6, 6, 0, 0), (7, 7, 0, 0), (7, 9, 0, 2), (8, 8, 0, 0)]

Subcase $(g,g') = (3,7)$ and $C'(\mathbb{F}_2) > 1$.

In [323]:
[(a[0], a[1], b[0], b[1]) for (a,b,_) in candidates[3,3,7] if b[0] > 1]

[(6, 10, 2, 2)]

Subcase $(g,g') = (3,7)$ if $C'(\mathbb{F}_2) = 1$.

In [324]:
[(a[0], a[1], a[2], b[0], b[1], b[2]) for (a,b,_) in candidates[3,3,7] if b[0] == 1 and b[1] >= 3]

[(5, 9, 14, 1, 3, 16)]

In [325]:
splitting_sequences([5,9,14],[1,3,16],3)

[[([3], [3], [3], [3], [2, 1]), ([3], [3]), ([3], [3], [2, 1])]]

In [326]:
assert not weil_polys_from_traces(P, 4, 2, [-3, -5, 0])

Subcase $(g,g') = (3,7)$ if $C'(\mathbb{F}_2) = 0$.

In [327]:
[(a[0], a[1], b[0], b[1]) for (a,b,_) in candidates[3,3,7] if b[0] == 0]

[(2, 8, 0, 0),
 (3, 7, 0, 0),
 (3, 7, 0, 0),
 (4, 6, 0, 0),
 (4, 6, 0, 0),
 (4, 8, 0, 0),
 (4, 8, 0, 0),
 (4, 8, 0, 0),
 (4, 10, 0, 2),
 (4, 12, 0, 6)]

In [328]:
assert not weil_polys_from_traces(P, 4, 2, [-2, -8])

In [329]:
assert not weil_polys_from_traces(P, 4, 2, [-3, -7])

In [330]:
assert not weil_polys_from_traces(P, 4, 2, [-4, -6])

In [331]:
assert not weil_polys_from_traces(P, 4, 2, [-4, -8])

Subcase $(g,g') = (2,4)$.

In [332]:
[(a[0], b[0]) for (a,b,_) in candidates[3,2,4]]

[(1, 0), (3, 0), (4, 0)]

Subcase $(g,g') = (2,6)$.

In [333]:
[(a[0], b[0], b[1]) for (a,b,_) in candidates[3,2,6]]

[(3, 0, 2), (4, 0, 2), (4, 0, 2), (5, 0, 2)]

Case $d=4$.

Show that $G \neq D_4$ by looking for iterated extensions.

In [334]:
for (d, g, g1) in [(4,3,9), (4,2,5)]:
    for (a,b,_) in candidates[d,g,g1]:
        for i in range(len(df)):
            if df.loc[i, "d"] == 2 and df.loc[i, "g"] == g and \
            df.loc[i, "g'"] == 2*g-1 and eval(df.loc[i, "Counts of C"]) == a and \
            df.loc[i, "Cyclic"] in ["Yes", "Unknown"]:
                for j in range(len(df)):
                    if df.loc[j, "d"] == 2 and df.loc[j, "g"] == 2*g-1 and \
                    df.loc[j, "g'"] == g1 and df.loc[j, "Cyclic"] in ["Yes", "Unknown"] and\
                    df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                    eval(df.loc[j, "Counts of C'"]) == b:
                        print(d,g,g1,a,b)

4 2 5 [2, 8, 8, 24, 52, 56, 100, 256, 476, 968, 2180, 4272, 8140] [0, 0, 12, 16, 60, 24, 84, 160, 444, 840, 2244, 4144, 8268]


Subcase $(g,g') = (3,9)$ and the quadratic resolvent is constant.

In [335]:
[(a[0], a[1], b[0], b[1]) for (a,b,c) in candidates[4,3,9] if c%3 == 0]

[(5, 9, 0, 0)]

Subcase $(g,g') = (2,5)$ and the quadratic resolvent is constant.

In [337]:
[(a[0], b[0]) for (a,b,c) in candidates[4,2,5] if c%3 == 0]

[(4, 1), (5, 1)]

Subcase $(g,g') = (3,9)$ and the quadratic resolvent is purely geometric.

In [336]:
[(a[0], b[0], b[1], b[2]) for (a,b,_) in candidates[4,3,9]]

[(5, 0, 0, 0), (6, 0, 0, 6), (6, 0, 2, 0), (6, 0, 0, 0), (6, 0, 2, 0)]

Subcase $(g,g') = (2,5)$ and the quadratic resolvent is purely geometric.

In [338]:
[(a[0], a[1], b[0], b[1], b[2], c) for (a,b,c) in candidates[4,2,5] if c%2 == 0]

[(2, 8, 0, 0, 12, 4),
 (4, 4, 0, 4, 12, 8),
 (4, 4, 1, 1, 10, 8),
 (4, 8, 0, 8, 6, 10),
 (4, 8, 1, 5, 4, 10)]

Case $d=5$.

In [339]:
[(a[:6], b[:6], c) for (a,b,c) in candidates[5,2,6]]

[([3, 5, 9, 33, 33, 65], [0, 0, 0, 20, 15, 90], 5),
 ([3, 7, 9, 31, 33, 43], [0, 0, 9, 8, 30, 33], 6),
 ([4, 8, 10, 24, 14, 56], [0, 0, 15, 20, 20, 45], 10),
 ([4, 8, 10, 24, 14, 56], [0, 6, 0, 18, 0, 60], 10),
 ([4, 10, 7, 18, 24, 55], [0, 2, 15, 18, 40, 23], 11),
 ([5, 9, 5, 17, 25, 81], [0, 6, 3, 34, 30, 63], 15),
 ([5, 9, 5, 17, 25, 81], [0, 10, 0, 10, 25, 70], 15),
 ([6, 6, 9, 10, 36, 87], [1, 3, 7, 27, 41, 69], 19)]

Assuming that $d=5$, convert a sequence of splittings to the traces of $B_1$ and $B_2$.

In [344]:
def splittings_to_B1_B2(ct1, ct2, s, return_both=False):
    d = {(5,): [(5,1), (5,) * 4],
         (3,1,1): [(3,3), (3,) * 6 + (1,) * 2],
         (2,2,1): [(2,) * 2 + (1,) * 2, (2,) * 10],
         (1,1,1,1,1): [(1,) * 6, (1,) * 20]}
    n = len(s)
    ct1p = place_count_from_point_count(ct1, n)
    ct2p = place_count_from_point_count(ct2, n)
    ct3p = [0 for _ in range(n)]
    ct4p = [0 for _ in range(n)]
    for i in range(n):
        for p in s[i]:
            pt = tuple(p)
            for j in d[pt][0]:
                if (i+1)*j <= n:
                    ct3p[(i+1)*j-1] += 1
            for j in d[pt][1]:
                if (i+1)*j <= n:
                    ct4p[(i+1)*j-1] += 1
    ct3 = point_count_from_place_count(ct3p, n)
    ct4 = point_count_from_place_count(ct4p, n)
    trA = [ct1[i] - ct2[i] for i in range(n)]
    trB1 = [ct1[i] - ct3[i] for i in range(n)]
    trB2 = [ct1[i] - ct4[i] - 2*trA[i] - trB1[i] for i in range(n)]
    return trB1, trB2

Subcase $(\#C(\mathbb{F}_2); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{32})) = (4; 0,6,0,18,0)$.

In [340]:
splitting_sequences([4,8,10,24,14],[0,6,0,18,0],5)

[]

Subcase $(\#C(\mathbb{F}_2), \dots, \#C(\mathbb{F}_8); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_8)) = (4,10,7;0,2,15)$,

In [341]:
splitting_sequences([4,10,7],[0,2,15],5,alternating=True)

[[([5], [5], [5], [5]), ([5], [5], [2, 2, 1]), ([1, 1, 1, 1, 1],)]]

Subcase $(\#C(\mathbb{F}_2), \#C(\mathbb{F}_4); \#C'(\mathbb{F}_2), \#C'(\mathbb{F}_4)) = (6,6;1,3)$.

In [342]:
splitting_sequences([6,6],[1,3],5,alternating=True)

[]

Subcase $(\#C(\mathbb{F}_2), \dots, \#C(\mathbb{F}_8); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_8)) = (5,9,5;0,6,3)$.

In [343]:
splitting_sequences([5,9,5],[0,6,3],5,alternating=True)

[]

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{32}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{32}))  = (3, 5, 9, 33, 33; 0, 0, 0, 20, 15)$.

In [410]:
l = splitting_sequences([3,5,9,33,33],[0,0,0,20,15],5,alternating=True)
print(set(s[0] for s in l))
print(set(s[1] for s in l))
print(set(s[2] for s in l))
print(set(s[4] for s in l))

{([5], [5], [5])}
{([5],)}
{([5], [5])}
{([5], [5], [5], [5], [5], [5])}


In [411]:
[splittings_to_B1_B2([3,5,9,33,33], [0,0,0,20,15], s) for s in l]

[([0, 0, 0, -20, -15], [-3, -5, -9, -53, -48]),
 ([0, 0, 0, 4, -15], [-3, -5, -9, -13, -48]),
 ([0, 0, 0, -8, -15], [-3, -5, -9, 7, -48]),
 ([0, 0, 0, -20, -15], [-3, -5, -9, 27, -48])]

In [347]:
set(trace_from_weil_poly(u, 5)[4] for u in weil_polys_from_traces(P, 12, 2, [-3, -5, -9]))

{-13, -8, -3, 2, 7, 12, 17, 22, 27, 32, 37, 47, 57}

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{8}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{8}))  = (4, 8, 10; 0, 0, 15)$,

In [348]:
set(s[0] for s in splitting_sequences([4,8,10],[0,0,15],5))

{([5], [5], [5], [5])}

In [349]:
l = splitting_sequences([4,8,10],[0,0,15],5,alternating=True)
l

[[([5], [5], [5], [5]), ([5], [5]), ([5], [1, 1, 1, 1, 1])]]

In [350]:
[splittings_to_B1_B2([4,8,10],[0,0,15], s) for s in l]

[([0, 0, -15], [-4, -8, -25])]

In [351]:
assert not weil_polys_from_traces(P, 12, 2, [-4,-8,-25])

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{8}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{8}))  = (5, 9, 5; 0, 10, 0)$.

In [352]:
splitting_sequences([5,9,5],[0,10,0],5)

[[([5], [5], [5], [5], [5]), ([5], [1, 1, 1, 1, 1]), ()],
 [([5], [5], [5], [5], [5]), ([3, 2], [1, 1, 1, 1, 1]), ()],
 [([5], [5], [5], [5], [5]), ([3, 1, 1], [2, 1, 1, 1]), ()]]

In [353]:
l = splitting_sequences([5,9,5],[0,10,0],5,alternating=True)
l

[[([5], [5], [5], [5], [5]), ([5], [1, 1, 1, 1, 1]), ()]]

In [354]:
[splittings_to_B1_B2([5,9,5],[0,10,0], s) for s in l]

[([0, -10, 0], [-5, -19, -5])]

In [355]:
assert not weil_polys_from_traces(P, 12, 2, [-5,-19,-5])

Subcase $(\#C(\mathbb{F}_2), \dots, \#C(\mathbb{F}_{128}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{128})) = (3,7,9,31,33,43,129;0,0,9,8,30,33,168)$.

In [356]:
l = splitting_sequences([3,7,9],[0,0,9],5)
set([s[0] for s in l])

{([5], [5], [5])}

In [357]:
l = splitting_sequences([3,7,9],[0,0,9],5,alternating=True)
l

[[([5], [5], [5]), ([5], [5]), ([3, 1, 1], [2, 2, 1])]]

In [358]:
[splittings_to_B1_B2([3,7,9],[0,0,9], s) for s in l]

[([0, 0, 0], [-3, -7, 3])]

In [359]:
l = splitting_sequences([3,7,9,31,33],[0,0,9,8,30],5,alternating=True)
l

[[([5], [5], [5]),
  ([5], [5]),
  ([3, 1, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [5], [3, 1, 1]),
  ([5], [5], [5], [5], [3, 1, 1], [2, 2, 1])],
 [([5], [5], [5]),
  ([5], [5]),
  ([3, 1, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [5], [3, 1, 1]),
  ([5], [5], [5], [2, 2, 1], [2, 2, 1], [2, 2, 1])],
 [([5], [5], [5]),
  ([5], [5]),
  ([3, 1, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [2, 2, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [3, 1, 1], [2, 2, 1])],
 [([5], [5], [5]),
  ([5], [5]),
  ([3, 1, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [2, 2, 1], [2, 2, 1]),
  ([5], [5], [5], [2, 2, 1], [2, 2, 1], [2, 2, 1])]]

In [360]:
[splittings_to_B1_B2([3,7,9,31,33],[0,0,9,8,30], s) for s in l]

[([0, 0, 0, 4, -15], [-3, -7, 3, -27, -28]),
 ([0, 0, 0, 4, -30], [-3, -7, 3, -27, -3]),
 ([0, 0, 0, -8, -15], [-3, -7, 3, -7, -28]),
 ([0, 0, 0, -8, -30], [-3, -7, 3, -7, -3])]

In [361]:
assert not weil_polys_from_traces(P, 12, 2, [-3,-7,3,-27,-28])

In [362]:
assert not weil_polys_from_traces(P, 12, 2, [-3,-7,3,-27,-3])

In [363]:
assert not weil_polys_from_traces(P, 12, 2, [-3,-7,3,-7,-28])

In [364]:
[trace_from_weil_poly(u,7) for u in weil_polys_from_traces(P, 12, 2, [-3,-7,3,-7,-3])]

[[-3, -7, 3, -7, -3, -19, 25],
 [-3, -7, 3, -7, -3, -13, 4],
 [-3, -7, 3, -7, -3, -7, -17]]

In [365]:
l2 = [trace_from_weil_poly(u,7) for u in weil_polys_from_traces(P, 10, 2, [0,0,0,-8,-30])]
l2

[[0, 0, 0, -8, -30, -24, 0]]

In [366]:
l = splitting_sequences([3,7,9,31,33,43,129],[0,0,9,8,30,33,168],5,alternating=True)
len(l)

248

In [367]:
l3 = [s for s in l if splittings_to_B1_B2([3,7,9,31,33,43,129],[0,0,9,8,30,33,168],s)[0] in l2]
len(l3)

2

In [407]:
[splittings_to_B1_B2([3,7,9,31,33,43,129],[0,0,9,8,30,33,168],s) for s in l3]

[([0, 0, 0, -8, -30, -24, 0], [-3, -7, 3, -7, -3, -19, -45]),
 ([0, 0, 0, -8, -30, -24, 0], [-3, -7, 3, -7, -3, -19, 95])]

Case $d=6$.

Assuming $d=6$, convert a splitting sequence into the traces of the Prym for the sextic twin. 
If `S6` is False, assume that $G \not\subseteq \mathrm{PGL}(2,5)$, so that the sextic twin becomes reducible.

In [414]:
def splittings_to_sextic_twin(ct1, s, S6=True):
    d = {(6,): [3,2,1],
         (3,2,1): [6],
         (4,1,1): [4,1,1],
         (2,2,2): [2,1,1,1,1],
         (2,1,1,1,1): [2,2,2],
         (5,1): [5,1],
         (4,2): [4,2],
         (3,3): [3,1,1,1],
         (3,1,1,1): [3,3],
         (2,2,1,1): [2,2,1,1],
         (1,1,1,1,1,1): [1,1,1,1,1,1]}
    n = len(ct1)
    ct2p = [0 for _ in range(n)]
    for i in range(n):
        for p in s[i]:
            pt = tuple(p)
            for j in d[pt]:
                if (i+1)*j <= n:
                    ct2p[(i+1)*j-1] += 1
    ct2 = point_count_from_place_count(ct2p, n)
    if S6:
        return [ct1[i] - ct2[i] for i in range(n)]
    return [2*ct1[i] - ct2[i] for i in range(n)]

Check for intermediate subfields.

In [369]:
for (a,b,_) in candidates[6,2,7]:
    for i in range(len(df)):
        if df.loc[i, "d"] == 2 and df.loc[i, "g"] == 2 and \
        df.loc[i, "g'"] == 3 and eval(df.loc[i, "Counts of C"]) == a and \
        df.loc[i, "Cyclic"] == "Yes":
            for j in range(len(df)):
                if df.loc[j, "d"] == 3 and df.loc[j, "g"] == 3 and \
                df.loc[j, "g'"] == 7 and df.loc[j, "Cyclic"] == "Yes" and\
                df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                eval(df.loc[j, "Counts of C'"]) == b:
                    print(a,b)

Remove subcases based on the reduced resultant.

In [371]:
l = []
for (a,b,c) in candidates[6,2,7]:
    u1 = weil_poly_from_point_count(a, 2)
    u2 = weil_poly_from_point_count(b, 7)
    v1 = u1.trace_polynomial()[0]
    v2 = (u2//u1).trace_polynomial()[0]
    if modified_reduced_resultant(v1, v2)%3 != 0:
        l.append((a, b, c))
[(a[:6], b[:6], c) for (a,b,c) in l]

[([5, 5, 17, 9, 25, 65], [0, 0, 3, 12, 35, 87], 13),
 ([5, 9, 5, 17, 25, 81], [0, 2, 9, 2, 30, 71], 15),
 ([5, 9, 5, 17, 25, 81], [0, 4, 0, 12, 15, 106], 15),
 ([6, 6, 9, 10, 36, 87], [0, 0, 15, 20, 30, 87], 19)]

In [372]:
[(a[:6], b[:6], c) for (a,b,c) in candidates[6,2,7] if (a,b,c) not in l]

[([4, 8, 10, 24, 14, 56], [0, 0, 6, 8, 30, 24], 10),
 ([5, 5, 17, 9, 25, 65], [0, 0, 12, 4, 15, 90], 13),
 ([5, 5, 17, 9, 25, 65], [0, 2, 3, 10, 30, 47], 13),
 ([5, 7, 11, 15, 15, 91], [0, 2, 6, 10, 5, 116], 14),
 ([5, 9, 5, 17, 25, 81], [1, 1, 1, 1, 41, 49], 15),
 ([6, 6, 9, 10, 36, 87], [0, 2, 3, 14, 35, 131], 19),
 ([6, 6, 9, 10, 36, 87], [0, 4, 3, 12, 30, 91], 19)]

Subcases with the quadratic resolvent constant which require only consideration of splitting sequences.

In [373]:
splitting_sequences([5,5], [0, 2], 6, alternating=True)

[]

In [374]:
splitting_sequences([5,9], [1, 1], 6, alternating=True)

[]

In [375]:
splitting_sequences([6,6], [0, 2], 6, alternating=True)

[]

In [376]:
splitting_sequences([6,6], [0, 4], 6, alternating=True)

[]

In [377]:
splitting_sequences([4, 8,10], [0, 0, 6], 6, alternating=True)

[[([6], [6], [6], [6]), ([4, 2], [4, 2]), ([6], [4, 1, 1])],
 [([6], [6], [6], [6]), ([4, 2], [4, 2]), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6]), ([4, 2], [4, 2]), ([3, 2, 1], [3, 2, 1])],
 [([6], [6], [6], [6]), ([4, 2], [3, 3]), ([6], [4, 1, 1])],
 [([6], [6], [6], [6]), ([4, 2], [3, 3]), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6]), ([4, 2], [3, 3]), ([3, 2, 1], [3, 2, 1])],
 [([6], [6], [6], [6]), ([3, 3], [3, 3]), ([6], [4, 1, 1])],
 [([6], [6], [6], [6]), ([3, 3], [3, 3]), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6]), ([3, 3], [3, 3]), ([3, 2, 1], [3, 2, 1])]]

In [378]:
splitting_sequences([4, 8,10,24,14,56], [0, 0, 6, 8, 30,24], 6, alternating=True)

[]

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{32}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{32})) = (5,7,11, 15,15; 0, 2, 6, 10,5)$ and quadratic resolvent constant.

In [381]:
l = splitting_sequences([5,7,11,15], [0,2,6,10], 6, alternating=True)
set([s[0] for s in l])

{([6], [6], [6], [6], [6])}

In [382]:
l = splitting_sequences([5,7,11], [0,2,6], 6, alternating=True)
l

[[([6], [6], [6], [6], [6]), ([5, 1],), ([6], [4, 1, 1])],
 [([6], [6], [6], [6], [6]), ([5, 1],), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6], [6]), ([5, 1],), ([3, 2, 1], [3, 2, 1])]]

In [383]:
[splittings_to_sextic_twin([5,7,11], s) for s in l]

[[0, -10, -18], [0, -10, -27], [0, -10, -9]]

In [384]:
l = splitting_sequences([5,7,11,15], [0,2,6,10], 6, alternating=True)
set([s[2] for s in l]), set([s[3] for s in l])

({([3, 2, 1], [3, 2, 1]), ([4, 1, 1], [2, 2, 2]), ([6], [4, 1, 1])},
 {([3, 3], [2, 2, 1, 1]), ([4, 2], [2, 2, 1, 1]), ([5, 1], [5, 1])})

In [385]:
[splittings_to_sextic_twin([5,7,11,15], s) for s in l]

[[0, -10, -18, -10],
 [0, -10, -18, -10],
 [0, -10, -18, -22],
 [0, -10, -27, -10],
 [0, -10, -27, -10],
 [0, -10, -27, -22],
 [0, -10, -9, -10],
 [0, -10, -9, -10],
 [0, -10, -9, -22]]

In [386]:
[trace_from_weil_poly(u,4)[3] for u in weil_polys_from_traces(P, 10, 2, [0,-10,-18])]

[18]

In [387]:
[trace_from_weil_poly(u,4)[3] for u in weil_polys_from_traces(P, 10, 2, [0,-10,-9])]

[-6, -2, 2, 6, 10]

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{64}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{64})) = (5,5,17,9,25,65; 0,0,12,4,15,90)$ and quadratic resolvent constant.

In [433]:
l = splitting_sequences([5,5,17,9], [0,0,12,4], 6, alternating=True)
set([s[0] for s in l])

{([6], [6], [6], [6], [6])}

In [434]:
l = splitting_sequences([5,5,17,9], [0,0,12,4], 6, alternating=True)
l = [s for s in l if [2,2,2] not in s[2] and [2,1,1,1,1] not in s[2]]
print(set([s[1] for s in l]))
print(set([s[2] for s in l]))
print(set([s[3] for s in l]))

{()}
{([6], [4, 1, 1], [3, 2, 1], [3, 2, 1]), ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]), ([6], [6], [4, 1, 1], [4, 1, 1])}
{([5, 1],)}


In [390]:
[splittings_to_sextic_twin([5,5,17,9], s) for s in l]

[[0, -10, -21, -10], [0, -10, -12, -10], [0, -10, -3, -10]]

In [391]:
assert not weil_polys_from_traces(P, 10, 2, [0,-10,-21,-10])

In [392]:
assert not weil_polys_from_traces(P, 10, 2, [0,-10,-12,-10])

In [393]:
[trace_from_weil_poly(u,5)[4] for u in weil_polys_from_traces(P, 10, 2, [0,-10,-3,-10])]

[10, 15]

In [394]:
l = splitting_sequences([5,5,17,9,25], [0,0,12,4,15], 6, alternating=True)
l = [s for s in l if [2,2,2] not in s[2] and [2,1,1,1,1] not in s[2] and [6] not in s[2]]
l

[[([6], [6], [6], [6], [6]),
  (),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]),
  ([5, 1],),
  ([6], [6], [4, 1, 1], [3, 2, 1])],
 [([6], [6], [6], [6], [6]),
  (),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]),
  ([5, 1],),
  ([6], [4, 1, 1], [3, 2, 1], [2, 2, 2])],
 [([6], [6], [6], [6], [6]),
  (),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]),
  ([5, 1],),
  ([6], [3, 2, 1], [3, 2, 1], [3, 2, 1])],
 [([6], [6], [6], [6], [6]),
  (),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]),
  ([5, 1],),
  ([4, 1, 1], [3, 2, 1], [2, 2, 2], [2, 2, 2])],
 [([6], [6], [6], [6], [6]),
  (),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]),
  ([5, 1],),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [2, 2, 2])]]

In [395]:
set([splittings_to_sextic_twin([5,5,17,9,25], s)[4] for s in l])

{-30, -15, 0, 15}

In [396]:
[trace_from_weil_poly(u,6) for u in weil_polys_from_traces(P, 10, 2, [0,-10,-3,-10,15])]

[[0, -10, -3, -10, 15, 23]]

In [397]:
l = splitting_sequences([5,5,17,9,25,65], [0,0,12,4,15,90], 6, alternating=True)
l = [s for s in l if splittings_to_sextic_twin([5,5,17,9,25],s) == [0,-10,-3,-10,15]]
len(l)

25

In [398]:
set([splittings_to_sextic_twin([5,5,17,9,25,65],s)[5] for s in l])

{-97, -79, -61, -43, -25, -7, 11, 29}

Subcases with the quadratic resolvent purely geometric.

In [399]:
counts = [([4,8,10,24,14],[0,0,6,8,30]), ([5,7,11,15,15],[0,2,6,10,5])]

In [429]:
for (ct1, ct2) in counts:
    print("Point counts: {}, {}".format(ct1, ct2))
    l = splitting_sequences(ct1[:3], ct2[:3], 6)
    l = [s for s in l if splittings_to_quadratic_resolvent(ct1[:3], s)[0] <= 2]
    print("Splittings in degree 1: {}".format(set(s[0] for s in l)))
    tmp = set(splittings_to_quadratic_resolvent(ct1[:3], s)[0] for s in l)
    assert len(tmp) == 1
    trB = list(tmp)[0]
    tmp = weil_polys_from_traces(P, 2, 2, [trB])
    assert len(tmp) == 1
    trB = trace_from_weil_poly(tmp[0], 4)
    print("Traces of B: {}".format(trB))
    l = [s for s in l if splittings_to_quadratic_resolvent(ct1[:3], s) == tuple(trB[:3])]
    print("Splittings in degree 2: {}".format(set(s[1] for s in l)))
    print("Splittings in degree 3: {}".format(set(s[2] for s in l)))
    trB1 = [splittings_to_sextic_twin(ct1[:3], s) for s in l]
    print("Traces of B1 (based on splitting): {}".format([set(t[i] for t in trB1) for i in range(3)]))
    trB1prime = [splittings_to_sextic_twin(ct1[:3], s, S6=False) for s in l]
    print("Traces of B1' (based on splitting): {}".format([set(t[i] for t in trB1prime) for i in range(3)]))
    trB1 = [trace_from_weil_poly(t, 5) for t1 in trB1 for t in weil_polys_from_traces(P, 10, 2, t1)]
    print("Traces of B1 (by exhaustion): {}".format([set(t[i] for t in trB1) for i in range(3)]))
    trB1prime = [trace_from_weil_poly(t, 5) for t1 in trB1prime for t in weil_polys_from_traces(P, 8, 2, t1)]
    print("Traces of B1' (by exhaustion): {}".format([set(t[i] for t in trB1prime) for i in range(3)]))
    tmp = [s for s in splitting_sequences(ct1[:4], ct2[:4], 6, seed=l)]
    l1 = [s for s in tmp if splittings_to_sextic_twin(ct1[:4], s) in [t[:4] for t in trB1]]
    tmp = [s for s in splitting_sequences(ct1[:4], ct2[:4], 6, seed=l, avoid=[[3,2,1],[4,2],[2,1,1,1,1],[3,1,1,1]])]
    l1prime = [s for s in tmp if splittings_to_sextic_twin(ct1[:4], s, S6=False) in [t[:4] for t in trB1prime]]
    print("Splittings in degree 4 (S6): {}".format(set([s[3] for s in l1])))
    print("Splittings in degree 4 (PGL25): {}".format(set([s[3] for s in l1prime])))
    tmp = [s for s in splitting_sequences(ct1[:5], ct2[:5], 6, seed=l)]
    l1 = [s for s in tmp if splittings_to_sextic_twin(ct1[:5], s) in trB1]
    tmp = [s for s in splitting_sequences(ct1[:5], ct2[:5], 6, seed=l, avoid=[[3,2,1],[4,2],[2,1,1,1,1],[3,1,1,1]])]
    l1prime = [s for s in tmp if splittings_to_sextic_twin(ct1[:5], s, S6=False) in trB1prime]
    assert not l1
    print("Splittings in degree 5 (S6): {}".format(set(s[4] for s in l1)))
    assert not l1prime
    print("Splittings in degree 5 (PGL25): {}".format(set(s[3] for s in l1prime)))
    print("This case complete!")
    print("")

Point counts: [4, 8, 10, 24, 14], [0, 0, 6, 8, 30]
Splittings in degree 1: {([6], [6], [6], [3, 3])}
Traces of B: [2, 0, -4, -8]
Splittings in degree 2: {([6], [6]), ([6], [2, 2, 2]), ([2, 2, 2], [2, 2, 2])}
Splittings in degree 3: {([4, 2], [3, 3]), ([4, 2], [4, 2]), ([3, 3], [3, 3])}
Traces of B1 (based on splitting): [{-2}, {-8, -14, -20}, {-8, -26, -17}]
Traces of B1' (based on splitting): [{2}, {0, -6, -12}, {-16, -7, 2}]
Traces of B1 (by exhaustion): [{-2}, {-8}, {-8}]
Traces of B1' (by exhaustion): [{2}, {0, -6, -12}, {-16, -7, 2}]
Splittings in degree 4 (S6): {([5, 1], [4, 2], [4, 2], [3, 2, 1]), ([4, 2], [4, 2], [3, 2, 1], [3, 2, 1]), ([6], [4, 2], [3, 2, 1], [3, 2, 1])}
Splittings in degree 4: {([6], [4, 1, 1], [3, 3], [3, 3]), ([6], [5, 1], [5, 1], [2, 2, 2]), ([5, 1], [5, 1], [3, 3], [2, 2, 2]), ([6], [6], [2, 2, 2], [2, 2, 1, 1]), ([6], [6], [6], [4, 1, 1]), ([6], [3, 3], [3, 3], [2, 2, 1, 1]), ([5, 1], [5, 1], [3, 3], [3, 3]), ([6], [6], [4, 1, 1], [2, 2, 2]), ([6], [6], 

Case $d=7$.

In [401]:
(d, g, g1) = (7, 2, 8)
candidates[d, g, g1]

[([5, 7, 11, 15, 15, 91, 131, 255, 479, 987, 2227, 3999, 8143],
  [0, 0, 0, 0, 0, 84, 133, 336, 378, 980, 2310, 4032, 8008],
  14)]

In [402]:
u = weil_poly_from_point_count(candidates[d, g, g1][0][0], g); u

T^4 + 2*T^3 + 3*T^2 + 4*T + 4

In [403]:
v = weil_poly_from_point_count(candidates[d, g, g1][0][1], g1); v

T^16 - 3*T^15 + 2*T^14 + 14*T^10 - 23*T^9 + 13*T^8 - 46*T^7 + 56*T^6 + 128*T^2 - 384*T + 256

In [404]:
K1 = magma.NumberField(u)
a  = K1.gen(1)
O1 = K1.MaximalOrder()
O1a = O1.sub([a, 2/a])
assert O1a == O1
assert O1.ClassNumber() == 1

In [405]:
K2 = magma.NumberField(v//u)
b = K2.gen(1)
O2 = K2.MaximalOrder()
O2a = O2.sub([b, 2/b])
assert O2a == O2
assert O2.ClassNumber() == 1

In [406]:
K2s = K2.sage()
l = K2s.primes_above(7)
assert(len(l) == 1)
p = l[0]
K2s(7).valuation(p)

6